<img src="https://images.pexels.com/photos/2803160/pexels-photo-2803160.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=750&w=1260" style="height: 650px" />
Fonte: <a href="https://www.pexels.com/pt-br/foto/adulto-ao-ar-livre-atividade-fisica-atleta-2803160/" target="_blank">Pexels.com<a>

# Introdução

Este projeto tem por objetivo efetuar análise exploratório e aplicar técnicas de machine learning para encontrar padrões e coisas interessantes sobre um conjunto de dados pessoal.

Esse conjunto de dados representa dados de corridas de rua realizadas por mim.

Os dados foram extraídos manualmente a partir do aplicativo MiFit que armazena os dados de exercícios coletados pelo smartwatch Amazfit Bip da Xiaomi.

# Imports

In [83]:
import pandas as pd
import numpy as np
import time

%matplotlib inline

# Leitura dos dados

In [12]:
df = pd.read_csv('data/dados.csv')
df.head()

,start_time,distance,duration,avg_pace,calories,avg_heart_rate,cadence,step,min_pace,max_heart_rate,km_min_pace,5km_pace
0,2019-11-02 17:28,3.52,00:19:57,00:05:39,266,153,159,99,00:05:09,178,1,NaN
1,2019-11-05 16:38,5.10,00:30:24,00:05:57,375,154,164,92,00:05:37,180,2,00:05:55
2,2019-11-08 17:52,4.09,00:23:16,00:05:41,296,157,162,96,00:05:21,180,3,NaN
3,2019-11-11 17:29,5.51,00:30:35,00:05:32,422,167,165,96,00:05:20,179,1,00:05:50
4,2019-11-13 17:00,5.37,00:30:23,00:05:39,393,156,162,99,00:05:29,177,3,00:05:47


## Dicionário de dados

- **start_time**: Data/hora em que a corrida foi iniciada.
- **distance**: Distância percorrida em KM.
- **duration**: Duração do percurso.
- **avg_pace**: Ritmo médio, ou seja, quantos minutos em média levou para completar 1Km.
- **calories**: Calorias gastas.
- **avg_heart_rate**: Frequência cardíaca média.
- **cadence**: Cadência é a frequência média de passos dados por minuto em uma corrida.
- **step**: Passada.
- **min_pace**: Pace/ritmo mais rápido.
- **max_heart_rate**: Frequência cardíaca máxima.
- **km_min_pace**: Km que teve o menor pace/ritmo.
- **5km_pace**: Pace referente ao 5Km.

## Informações sobre as variáveis

### Cadência

É a frequência de passos dados em uma corrida. Basta que você conte os seus passos no intervalo de 1 minuto para encontrar o valor atual da sua. Esse número tende a ser algo entre 160 e 190 passos, sendo que é comum os atletas considerarem o número de 180 ppm (passos por minuto) o ideal.

Fonte: <a href="https://corridaperfeita.com/cadencia-na-corrida/">Corrida perfeita - Acerte a cadência na corrida para correr melhor</a>

### Passada

É a distância em centímetros da pisada no chão de um mesmo pé.

# Perguntas

Nessa fase são elencadas quais as perguntas que devem ser respondidas através dos dados coletados.

## Cadência

- Quando a cadência foi maior o desempenho foi melhor, e vice-versa?
- Em que momentos do dia a cadência foi melhor?

## Passada

- Qual relação da passada com o ritmo menor/maior?

## Ritmo

- Há diferença de desempenho nos períodos manhã, tarde e noite?
- Em que momentos do dia o desempenho foi melhor?
- O pace/ritmo médio está evoluindo?
- Qual o pace ideal para corridas de 5K?
- É possível prever em quanto tempo chegarei num pace ideal?
- Qual o Km que tem o melhor pace?
- Como está a evolução do pace nos 5Km?
- Qual a tendência de ritmo/pace do início da corrida até o fim?

## Frequência cardíaca

- Qual a relação da frequência cardíaca com pace menor/maior?

# Limpeza dos dados

Nessa fase são aplicadas limpezas e tratamento de dados de acordo com as necessidades.

## Transformações

Várias colunas estão com formatos texto para representar os tempos e devem ser formatadas para números.

**Funções para transformação**

In [62]:
def transform_time_into_duration_seconds(time):
    """
    Tranform time string into duration in seconds.
    
    Attributes:
    ----------
    time : str
        time string that needs to be in format "HH:MM:SS"
    
    Returns
    -------
    int
        time duration in seconds      
    """
    if isinstance(time, str):
        hours, minutes, seconds = map(int, time.split(":"))
        duration = (hours * 3600) + (minutes * 60) + seconds
        return duration


def transform_duration_seconds_into_time(seconds):
    """
    Tranform durantion seconds into time.
    
    Attributes:
    ----------
    seconds : int
        duration in seconds
    
    Returns
    -------
    str
        time in format "HH:MM:SS"     
    """    
    hours, minutes, seconds = map(int, time.split(":"))
    duration = (hours * 3600) + (minutes * 60) + seconds
    return time

In [82]:
seconds = 3660
hours = seconds // 3600
minutes = seconds // 60
seconds = seconds % 60
time = "{}:{}:{}".format(str(hours).zfill(2), str(minutes).zfill(2), str(seconds).zfill(2))
print(time)

01:61:00


In [87]:
time.strftime('%H:%M:%S', time.gmtime(3680))

'01:01:20'

**Aplicando a transformação**

In [60]:
columns = ['duration', 'avg_pace', 'min_pace', '5km_pace']
for column in columns:
    df[column+'_sec'] = df[column].map(lambda value: transform_time_into_duration_seconds(value))

In [63]:
df.head()

,start_time,distance,duration,avg_pace,calories,avg_heart_rate,cadence,step,min_pace,max_heart_rate,km_min_pace,5km_pace,duration_sec,avg_pace_sec,min_pace_sec,5km_pace_sec
0,2019-11-02 17:28,3.52,00:19:57,00:05:39,266,153,159,99,00:05:09,178,1,NaN,1197,339,309,NaN
1,2019-11-05 16:38,5.10,00:30:24,00:05:57,375,154,164,92,00:05:37,180,2,00:05:55,1824,357,337,355.0
2,2019-11-08 17:52,4.09,00:23:16,00:05:41,296,157,162,96,00:05:21,180,3,NaN,1396,341,321,NaN
3,2019-11-11 17:29,5.51,00:30:35,00:05:32,422,167,165,96,00:05:20,179,1,00:05:50,1835,332,320,350.0
4,2019-11-13 17:00,5.37,00:30:23,00:05:39,393,156,162,99,00:05:29,177,3,00:05:47,1823,339,329,347.0


In [64]:
df['avg_pace_sec'].mean()

340.5

## Dados faltantes

In [41]:
df.isna().sum()

start_time         0
distance           0
duration           0
avg_pace           0
calories           0
avg_heart_rate     0
cadence            0
step               0
min_pace           0
max_heart_rate     0
km_min_pace        0
5km_pace          11
dtype: int64

A coluna **km_pace** tem linhas que possuem valor nulo, que são aquelas em que a distância percorrida não passou dos 5Km. 

Para estes casos, será atribuído a média dessa coluna, com base nas linhas que tem valor registrado.

In [ ]:
#...

# Análise Exploratória

Nessa fase são feitas as análise de forma a responder as perguntas elencadas anteriormente.

In [17]:
df.describe()

,distance,calories,avg_heart_rate,cadence,step,max_heart_rate,km_min_pace
count,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000
mean,5.003824,363.000000,158.558824,163.352941,100.941176,175.617647,2.176471
std,1.047511,82.826946,6.661473,4.205929,11.591379,3.884817,1.192669
min,2.510000,176.000000,134.000000,145.000000,81.000000,166.000000,1.000000
25%,4.427500,319.500000,155.000000,162.000000,96.000000,173.000000,1.000000
50%,5.350000,376.500000,159.000000,164.000000,98.000000,176.500000,2.000000
75%,5.697500,428.750000,162.750000,166.000000,100.000000,178.000000,3.000000
max,6.500000,509.000000,167.000000,168.000000,141.000000,184.000000,5.000000


CONTINUA...